In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import itertools

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 导入数据
-  label 文件一共有 79572 行
- 各种维度混合在一起
- 我们的目的是切分开各种维度, 进行训练和模拟
- 那么首先取出个标签的图片进行聚类

In [2]:
df_train = pd.read_csv('./Annotations/label.csv', header=None)
df_train.columns = ['image_id', 'class', 'label']
df_train.head()

,image_id,class,label
0,Images/collar_design_labels/4d8a38b29930a403e5...,collar_design_labels,nnynn
1,Images/collar_design_labels/bd0981f231180d2b00...,collar_design_labels,nynnn
2,Images/collar_design_labels/26937e1724feadfe39...,collar_design_labels,ynnnn
3,Images/collar_design_labels/cf4140ec542887270f...,collar_design_labels,nynnn
4,Images/collar_design_labels/50644b2b9de045f2d1...,collar_design_labels,nynnn


In [3]:
classes = ['collar_design_labels', 'neckline_design_labels', 'skirt_length_labels',
          'sleeve_length_labels', 'neck_design_labels', 'coat_length_labels', 'lapel_design_labels',
          'pant_length_labels']

In [4]:
cur_class = classes[7]
df_load = df_train[(df_train['class'] == cur_class)].copy()
df_load.reset_index(inplace=True)
del df_load['index']

print('{0}: {1}'.format(cur_class, len(df_load)))
df_load.head()

pant_length_labels: 7460


,image_id,class,label
0,Images/pant_length_labels/07264732b8dfa05ca485...,pant_length_labels,ynnnnn
1,Images/pant_length_labels/ae01c534cddd89261c95...,pant_length_labels,nnynnn
2,Images/pant_length_labels/7d51f1a23c93213e4652...,pant_length_labels,nnnnny
3,Images/pant_length_labels/5c9f7c834ae1c1daa3f5...,pant_length_labels,nynnnn
4,Images/pant_length_labels/e1eb7a176c228107125d...,pant_length_labels,nnnnyn


In [5]:
n = len(df_load)
n_class = len(df_load['label'][0])
width = 299 # 定义图片大小

X = np.zeros((n, width, width, 3), dtype=np.uint8)
y = np.zeros((n, n_class), dtype=np.uint8)

In [6]:
n_class

6

In [7]:
print('./{0}'.format(df_load['image_id'][0]))

./Images/pant_length_labels/07264732b8dfa05ca4857c71d2f9fb4b.jpg


In [8]:
for i in tqdm(range(n)):
    tmp_label = df_load['label'][i]
    if len(tmp_label) > n_class:
        print(df_load['image_id'][i])
    X[i] = cv2.resize(cv2.imread('./{0}'.format(df_load['image_id'][i])), (width, width))
    y[i][tmp_label.find('y')] = 1

100%|██████████| 7460/7460 [00:49<00:00, 151.79it/s]


In [9]:
X.shape, y.shape

((7460, 299, 299, 3), (7460, 6))

In [10]:
#plt.figure(figsize=(12, 7))
#for i in range(8):
#    random_index = random.randint(0, n-1)
#    plt.subplot(2, 4, i+1)
#    plt.imshow(X[random_index][:,:,::-1])
#    plt.title(y[random_index])

## 提取特征

In [7]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import *
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import *
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import RMSprop
import keras

Using TensorFlow backend.


In [8]:
cnn_model = InceptionV3(include_top=False, input_shape=(width, width, 3), weights='imagenet')

In [9]:
inputs = Input((width, width, 3))

x = inputs
x = Lambda(preprocess_input, name='preprocessing')(x)
x = cnn_model(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(n_class, activation='softmax', name='softmax')(x)

model = Model(inputs, x)

In [10]:
prefix_cls = cur_class.split('_')[0]
prefix_cls

'pant'

## 划分训练/测试集

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.12, random_state=42)
X_train.shape, y_train.shape

((6564, 299, 299, 3), (6564, 6))

In [16]:
# Compile the model
adam = Adam(lr=0.001) 

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
# Set a learning rate annealer
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
#                                             patience=3,
#                                             verbose=1,
#                                             factor=0.1,
#                                             min_lr=0.00001)
epochs = 16
batch_size = 32

## 数据增强

In [17]:
datagen = ImageDataGenerator(
        featurewise_center = False, # set input mean to 0 over the dataset
        samplewise_center = False, # set each sample mean to 0
        featurewise_std_normalization = False, # divide inputs by std of the dataset
        samplewise_std_normalization = False, # divide each input by its std
        zca_whitening = False, # apply ZCA whitening
        rotation_range = 10, # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.05, # randomly zoom image
        width_shift_range = 0.075, # randomly shift images horizontally (fraction of total width)
        height_shift_range = 0.075, # randomly shift images vertivally (fraction of total heigth)
        horizontal_flip = True, # randomly flip images
        vertical_flip = False,
        shear_range = 0.075,
        fill_mode = 'constant',
        cval = 0)

datagen.fit(X_train)


- 少量旋转
- 少量偏移
- 水平翻转
- 垂直翻转

In [18]:
prefix_cls = cur_class.split('_')[0]

checkpointer = ModelCheckpoint(filepath='./models/{0}.best0420_InceptionV3.h5'.format(prefix_cls), verbose=1, 
                               save_best_only=True, save_weights_only=True, mode='val_acc')

try:
    # Fit the model
    history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
                                  epochs=epochs, validation_data=(X_valid, y_valid),
                                  verbose=2, steps_per_epoch=X_train.shape[0] // batch_size,
                                  callbacks=[EarlyStopping(patience=5), checkpointer])
except KeyboardInterrupt:
    print('KeyboardInterrupt')

Epoch 1/16
 - 133s - loss: 1.0844 - acc: 0.5845 - val_loss: 2.1303 - val_acc: 0.4821

Epoch 00001: val_loss improved from inf to 2.13029, saving model to ./models/pant.best0420_InceptionV3.h5
Epoch 2/16
 - 127s - loss: 0.7450 - acc: 0.7255 - val_loss: 0.7877 - val_acc: 0.7254

Epoch 00002: val_loss improved from 2.13029 to 0.78769, saving model to ./models/pant.best0420_InceptionV3.h5
Epoch 3/16
 - 129s - loss: 0.6687 - acc: 0.7532 - val_loss: 0.7091 - val_acc: 0.7656

Epoch 00003: val_loss improved from 0.78769 to 0.70912, saving model to ./models/pant.best0420_InceptionV3.h5
Epoch 4/16
 - 128s - loss: 0.5780 - acc: 0.7829 - val_loss: 0.7647 - val_acc: 0.7288

Epoch 00004: val_loss did not improve
Epoch 5/16
 - 127s - loss: 0.5609 - acc: 0.7986 - val_loss: 0.5792 - val_acc: 0.7969

Epoch 00005: val_loss improved from 0.70912 to 0.57915, saving model to ./models/pant.best0420_InceptionV3.h5
Epoch 6/16
 - 125s - loss: 0.5264 - acc: 0.8105 - val_loss: 0.6251 - val_acc: 0.7902

Epoch 0000

In [19]:
cnn_model = InceptionV3(include_top=False, input_shape=(width, width, 3), weights='imagenet')

In [20]:
inputs = Input((width, width, 3))

x = inputs
x = Lambda(preprocess_input, name='preprocessing')(x)
x = cnn_model(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(n_class, activation='softmax', name='softmax')(x)

model = Model(inputs, x)

In [21]:
model.load_weights('./models/{}.best0420_InceptionV3.h5'.format(prefix_cls)) #加载以前训练好的模型，继续测试

In [22]:
# Compile the model
adam = Adam(lr=0.0001) 

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
# Set a learning rate annealer
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
#                                             patience=3,
#                                             verbose=1,
#                                             factor=0.1,
#                                             min_lr=0.00001)
epochs = 16
batch_size = 32

In [23]:
prefix_cls = cur_class.split('_')[0]

checkpointer = ModelCheckpoint(filepath='./models/{0}.best0420_InceptionV3.h5'.format(prefix_cls), verbose=1, 
                               save_best_only=True, mode='val_acc')

try:
    # Fit the model
    history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
                                  epochs=epochs, validation_data=(X_valid, y_valid),
                                  verbose=2, steps_per_epoch=X_train.shape[0] // batch_size,
                                  callbacks=[EarlyStopping(patience=5), checkpointer])
except KeyboardInterrupt:
    print('KeyboardInterrupt')

Epoch 1/16
 - 134s - loss: 0.3299 - acc: 0.8785 - val_loss: 0.3463 - val_acc: 0.8638

Epoch 00001: val_loss improved from inf to 0.34627, saving model to ./models/pant.best0420_InceptionV3.h5
Epoch 2/16
 - 129s - loss: 0.2564 - acc: 0.9024 - val_loss: 0.3323 - val_acc: 0.8728

Epoch 00002: val_loss improved from 0.34627 to 0.33229, saving model to ./models/pant.best0420_InceptionV3.h5
Epoch 3/16
 - 127s - loss: 0.2321 - acc: 0.9157 - val_loss: 0.3587 - val_acc: 0.8694

Epoch 00003: val_loss did not improve
Epoch 4/16
 - 126s - loss: 0.1989 - acc: 0.9244 - val_loss: 0.3530 - val_acc: 0.8817

Epoch 00004: val_loss did not improve
Epoch 5/16
KeyboardInterrupt


## 加载模型

In [11]:
df_test = pd.read_csv('./z_rank/Tests/question.csv', header=None)
df_test.columns = ['image_id', 'class', 'x']
del df_test['x']
df_test.head()

,image_id,class
0,Images/collar_design_labels/faad3490a16c7f3d4f...,collar_design_labels
1,Images/collar_design_labels/0b2b4254f35ce3a41a...,collar_design_labels
2,Images/collar_design_labels/7f2be608e06f804dd5...,collar_design_labels
3,Images/collar_design_labels/4b09d4dca80caac42e...,collar_design_labels
4,Images/collar_design_labels/de91f00a05e84d7239...,collar_design_labels


In [12]:
df_load = df_test[(df_test['class'] == cur_class)].copy()
df_load.reset_index(inplace=True)
del df_load['index']

print('{0}: {1}'.format(cur_class, len(df_load)))
df_load.head()

pant_length_labels: 1434


,image_id,class
0,Images/pant_length_labels/e78538758763e84e9700...,pant_length_labels
1,Images/pant_length_labels/1e680886dd3d65882745...,pant_length_labels
2,Images/pant_length_labels/7e25106e7a0f5ac5d26a...,pant_length_labels
3,Images/pant_length_labels/53fa07f57908d258f4ba...,pant_length_labels
4,Images/pant_length_labels/737319237b262a9515a5...,pant_length_labels


In [13]:
'./z_rank/{0}'.format(df_load['image_id'][0])

'./z_rank/Images/pant_length_labels/e78538758763e84e97009dda11754b47.jpg'

In [14]:
n = len(df_load)
X_test = np.zeros((n, width, width, 3), dtype=np.uint8)

for i in tqdm(range(n)):
    X_test[i] = cv2.resize(cv2.imread('./z_rank/{0}'.format(df_load['image_id'][i])), (width, width))


100%|██████████| 1434/1434 [00:08<00:00, 174.98it/s]


In [15]:
model.load_weights('./models/{}.best0420_InceptionV3.h5'.format(prefix_cls))

In [16]:
test_np = model.predict(X_test, batch_size=256)

In [17]:
test_np.shape

(1434, 6)

In [18]:
result = []

for i, row in df_load.iterrows():
    tmp_list = test_np[i]
    tmp_result = ''
    for tmp_ret in tmp_list:
        tmp_result += '{:.4f};'.format(tmp_ret)
        
    result.append(tmp_result[:-1])

df_load['result'] = result
df_load.head()

,image_id,class,result
0,Images/pant_length_labels/e78538758763e84e9700...,pant_length_labels,0.0000;0.0000;0.0000;0.0394;0.4119;0.5486
1,Images/pant_length_labels/1e680886dd3d65882745...,pant_length_labels,0.0062;0.0054;0.9855;0.0028;0.0001;0.0000
2,Images/pant_length_labels/7e25106e7a0f5ac5d26a...,pant_length_labels,0.0967;0.0045;0.1461;0.4598;0.1608;0.1320
3,Images/pant_length_labels/53fa07f57908d258f4ba...,pant_length_labels,0.0002;0.0001;0.0004;0.9885;0.0096;0.0013
4,Images/pant_length_labels/737319237b262a9515a5...,pant_length_labels,0.0000;0.0000;0.0000;0.0007;0.0834;0.9158


In [19]:
df_load.to_csv('./result/{}_0421a.csv'.format(prefix_cls), header=None, index=False)

In [20]:
df_load = pd.read_csv('./result/{}_0421a.csv'.format(prefix_cls), header=None)

In [21]:
len(df_load)

1434